In [1]:
!pip install --no-cache-dir transformers sentencepiece -q datasets

In [2]:
import pandas as pd
train_df=pd.read_csv("/content/augmented-with-paraphrasing-goemotions.csv")
valid_df=pd.read_csv("/content/goemotions_subset_val_new.csv")
test_df=pd.read_csv("/content/goemotions_subset_test_new.csv")

In [3]:
train_df.head()

,tweet,label
0,"हे बसणे खूप कठीण आहे, मला या डूडच्या शपथांवर व...",-1
1,कारण या उपमधले इडियट्स इतर लोकांच्या स्क्रीनवर...,-1
2,पाणी हा मानवी हक्क नाही. ते मूर्खपणाचे आहे.,-1
3,पातळ बाजू आणि अंतरेषा खरोखर मला फेकून देत आहेत.,-1
4,या पॉवरप्लेवर कोणतेही ध्येय नसल्यास निराश न हो...,-1


In [4]:
valid_df.head()

,Unnamed: 0.1,Unnamed: 0,text,three_point,marathi_text
0,0,33271,groceries are getting crazy expensive,-1,किराणा सामान महाग होत आहे
1,1,55535,You are not intereating. You are an annoying c...,-1,तुम्ही इंटरेस्ट करत नाही आहात. तू एक त्रासदायक...
2,2,27738,[NAME] doesn't believe in tipping.,-1,[NAME] टिपिंगवर विश्वास ठेवत नाही.
3,3,46073,"At this point, LSC only exists to give sociali...",-1,"या टप्प्यावर, LSC फक्त समाजवाद्यांना बदनाम करण..."
4,4,40593,I have two and nothing has ever happened to me...,-1,माझ्याकडे दोन आहेत आणि मला काहीही झाले नाही. म...


In [5]:
test_df.head()

,Unnamed: 0.1,Unnamed: 0,text,three_point,marathi_text
0,2,55326,It’s not really a surprise that sick people co...,-1,आजारी नसलेल्या लोकांपेक्षा आजारी लोक अधिक आरोग...
1,13,1503,They'd probably just use it to buy up armies a...,-1,ते कदाचित त्याचा उपयोग जुलूम करणार्‍यांसाठी सै...
2,15,32294,I didn't know [NAME] was a [NAME] Lord.,-1,[NAME] हे [NAME] प्रभु आहेत हे मला माहीत नव्हते.
3,18,3305,You shut your stupid mouth troll. Thy absolute...,-1,तू तुझं मूर्ख तोंड वेताळ बंद कर. तुझा एकदम फरक
4,22,37741,[NAME] damn you are autistic,-1,"[नाम] अरे, तू ऑटिस्टिक आहेस"


In [6]:
valid_df.drop(['Unnamed: 0.1', 'Unnamed: 0', 'text'], inplace=True, axis=1)
test_df.drop(['Unnamed: 0.1', 'Unnamed: 0', 'text'], inplace=True, axis=1)

In [7]:
dict1 = {'marathi_text': 'tweet', 'three_point': 'label'}
valid_df.rename(columns=dict1, inplace=True)
valid_df = valid_df.iloc[:, [1, 0]]
test_df.rename(columns=dict1, inplace=True)
test_df = test_df.iloc[:, [1, 0]]

In [8]:
train_df['label'] = train_df['label'].replace(1,2)
test_df['label'] = test_df['label'].replace(1,2)
valid_df['label'] = valid_df['label'].replace(1,2)

train_df['label'] = train_df['label'].replace(0,1)
test_df['label'] = test_df['label'].replace(0,1)
valid_df['label'] = valid_df['label'].replace(0,1)

train_df['label'] = train_df['label'].replace(-1,0)
test_df['label'] = test_df['label'].replace(-1,0)
valid_df['label'] = valid_df['label'].replace(-1,0)

In [9]:
# data preprocessing:
# removing non marathi text 
import re
def remove_non_marathi_goemotions(text):
    i= re.sub(r"http\S+", "", text)
    i= re.sub(r"pic\S+", "", i)
    i= re.sub(r"#", "", i)
    i= re.sub(r"lokmat\S+", "", i)
    i= re.sub(r"@\S+", "", i)
    i= re.sub(r"fb\S+", "", i)
    i= re.sub(r"twitter\S+", "", i)
    i= re.sub(r"bit\S+", "", i)    
    # i= re.sub(r"['\"!-./,;\[\]?]", "", i)  # during augmenting, keep the necessary punctuations [.?!'"] in the sentence.  
    i= re.sub(r"[\[\]]", "", i)    
    i= re.sub(r'[a-zA-Z0-9]', "", i)    
    i=re.split(r'[\s\n]+', i.strip())
    return(' '.join(i))

In [10]:
for i in range(len(valid_df)):
  valid_df['tweet'][i] = remove_non_marathi_goemotions(valid_df['tweet'][i])
for i in range(len(test_df)):
  test_df['tweet'][i] = remove_non_marathi_goemotions(test_df['tweet'][i])

<ipython-input-10-f547f5d1458d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['tweet'][i] = remove_non_marathi_goemotions(valid_df['tweet'][i])
<ipython-input-10-f547f5d1458d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['tweet'][i] = remove_non_marathi_goemotions(test_df['tweet'][i])


In [11]:
for i in range(len(train_df)):
  train_df['tweet'][i] = remove_non_marathi_goemotions(train_df['tweet'][i])

<ipython-input-11-309daa24fa48>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['tweet'][i] = remove_non_marathi_goemotions(train_df['tweet'][i])


In [12]:
train_df = train_df[train_df['tweet']!='']
valid_df = valid_df[valid_df['tweet']!='']
test_df = test_df[test_df['tweet']!='']

In [13]:
print(len(train_df))
print(len(valid_df))
print(len(test_df))

24982
5135
8998


In [14]:
train_df.drop_duplicates(inplace=True)
valid_df.drop_duplicates(inplace=True)
test_df.drop_duplicates(inplace=True)
train_df.dropna(inplace=True)
valid_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [15]:
print(len(train_df))
print(len(valid_df))
print(len(test_df))

24900
5134
8995


In [16]:
# shuffle the rows and reset the indices
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.sample(frac=1, random_state=42).reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [17]:
train_df

,tweet,label
0,छान! मला इतके दिवस त्यांच्या किंवा टूलचं ऐकलं ...,0
1,"तुम्ही उत्तर कोरिया, चीन किंवा स्वीडनसारख्या ध...",1
2,"""पेटा मंजूर"" मी बाहेर आहे. मी एमएलएमचा तिरस्का...",1
3,त्यांना पैसे मिळत नाहीत.,0
4,मला वाटतं की हे साल असेल ते खरंच कोसळायला सुरु...,2
...,...,...
24895,ती कदाचित असे करेल.,2
24896,नाम हे एक महान बाबा आहेत 😩,1
24897,"कडून अधिक असभ्यता, कुरूपता आणि लैंगिकता. जर ते...",0
24898,मी नाराज आहे आणि आधीच करतोय.,0


In [18]:
train_df.to_csv('/content/data/goemotions-train-new.csv')
test_df.to_csv('/content/data/goemotions-test-new.csv')
valid_df.to_csv('/content/data/goemotions-valid-new.csv')

In [19]:
train_df

,tweet,label
0,छान! मला इतके दिवस त्यांच्या किंवा टूलचं ऐकलं ...,0
1,"तुम्ही उत्तर कोरिया, चीन किंवा स्वीडनसारख्या ध...",1
2,"""पेटा मंजूर"" मी बाहेर आहे. मी एमएलएमचा तिरस्का...",1
3,त्यांना पैसे मिळत नाहीत.,0
4,मला वाटतं की हे साल असेल ते खरंच कोसळायला सुरु...,2
...,...,...
24895,ती कदाचित असे करेल.,2
24896,नाम हे एक महान बाबा आहेत 😩,1
24897,"कडून अधिक असभ्यता, कुरूपता आणि लैंगिकता. जर ते...",0
24898,मी नाराज आहे आणि आधीच करतोय.,0


In [20]:
from datasets import load_dataset
data_files={"train":"/content/data/goemotions-train-new.csv","test":"/content/data/goemotions-test-new.csv","validation":"/content/data/goemotions-valid-new.csv"}
dataset = load_dataset('/content/',data_files=data_files)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/content-0f7be25160f1be7f/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")

<ipython-input-21-4ff7ed1e6f07>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [22]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/marathi-bert-v2")

In [23]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

In [24]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [25]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=1)

In [26]:
def tokenize_function(examples):
    return tokenizer(examples["tweet"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/24900 [00:00<?, ? examples/s]

Map:   0%|          | 0/8995 [00:00<?, ? examples/s]

Map:   0%|          | 0/5134 [00:00<?, ? examples/s]

In [27]:
from transformers import AutoModelForSequenceClassification

In [28]:
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/marathi-bert-v2", num_labels=3)

Some weights of the model checkpoint at l3cube-pune/marathi-bert-v2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/marat

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

In [30]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.888200,0.845385,0.635177


TrainOutput(global_step=3113, training_loss=0.9257314984070079, metrics={'train_runtime': 2707.6111, 'train_samples_per_second': 9.196, 'train_steps_per_second': 1.15, 'total_flos': 6551524101427200.0, 'train_loss': 0.9257314984070079, 'epoch': 1.0})

In [31]:
predictions = trainer.predict(tokenized_datasets["test"])

In [32]:
preds = np.argmax(predictions.predictions, axis=-1)

In [33]:
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.6253474152306837}

In [34]:
def remove_non_marathi_in_tweets(text):
    i= re.sub(r"http\S+", "", text)
    i= re.sub(r"pic\S+", "", i)
    i= re.sub(r"#", "", i)
    i= re.sub(r"lokmat\S+", "", i)
    i= re.sub(r"@\S+", "", i)
    i= re.sub(r"fb\S+", "", i)
    i= re.sub(r"twitter\S+", "", i)
    i= re.sub(r"bit\S+", "", i)    
    i= re.sub(r"['\"!-./,;\[\]?]", "", i)  # during augmenting, keep the necessary punctuations [.?!'"] in the sentence.  
    i= re.sub(r"[\[\]]", "", i)    
    i= re.sub(r'[a-zA-Z0-9]', "", i)    
    i=re.split(r'[\s\n]+', i.strip())
    return(' '.join(i))

In [35]:
# prediction on MahaSent test set:
test_df_tweets=pd.read_csv("tweets-test.csv")
test_df_tweets['label'] = test_df_tweets['label'].replace(1,2)
test_df_tweets['label'] = test_df_tweets['label'].replace(0,1)
test_df_tweets['label'] = test_df_tweets['label'].replace(-1,0)
for i in range(len(test_df_tweets)):
  test_df_tweets['tweet'][i] = remove_non_marathi_in_tweets(test_df_tweets['tweet'][i])
test_df_tweets = test_df_tweets[test_df_tweets['tweet']!='']
test_df_tweets.drop_duplicates(inplace=True)
test_df_tweets.dropna(inplace=True)
test_df_tweets.to_csv('data/tweets-test.csv')
data_files_tweets={"test":"data/tweets-test.csv"}
dataset_tweets = load_dataset('/content/',data_files=data_files_tweets)
tokenized_datasets_tweets = dataset_tweets.map(tokenize_function, batched=True)

<ipython-input-35-0f012f4c5b86>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_tweets['tweet'][i] = remove_non_marathi_in_tweets(test_df_tweets['tweet'][i])


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/content-b4ad556312520c7c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2248 [00:00<?, ? examples/s]

In [36]:
predictions = trainer.predict(tokenized_datasets_tweets["test"])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.7210854092526691}

In [37]:
import pandas as pd
train_df_new=pd.read_csv("/content/augmented-with-paraphrasing-goemotions.csv")

In [38]:
train_df_new['tweet'][7]

'आजपर्यंत, मी त्यात नाम नाव असलेले काहीही वाचून घाबरून जातो'

In [39]:
train_df_new['tweet'][12498]

'आजपर्यंत मी त्यात नाव नाव असलेले काही वाचून घाबरुन जातो.'